# GSM model for C. glutamicum

In [1]:
import numpy as np
from cobra.io import read_sbml_model

We load the model of C. glutamicum

In [2]:
model = read_sbml_model('seed_Seed196627_4.COBRA-sbml3.xml')
model

Name,seed_Seed196627_4
Memory address,0x07fb37cbe2ac8
Number of metabolites,988
Number of reactions,1007
Number of groups,0
Objective expression,1.0*reacD3FA036D - 1.0*reacD3FA036D_reverse_06ea7
Compartments,"extracellular region, cytoplasm"


# Inserting enzyme 2.6.1.13

To make C. glutamicum produce putrescine via this new pathway we need to add the reactions of the enzymes 2.6.1.13 and 4.1.1.17

In [3]:
from cobra import Reaction, Metabolite

### 2.6.1.13

In [4]:
new_reaction_1 = Reaction('OAT')  # OAT -> (o)rnithine (a)mino(t)ransferase

We need to find the id's of the metabolites that are in this reaction: L-glutamate 5-semialdehyde, L-glutamate, Ornithine and 2-oxoglutarate

In [5]:
for metabolite in model.metabolites.query('L-glutamate 5-semialdehyde', 'name'):
    print(metabolite.name)
    print(metabolite.id)

L-glutamate 5-semialdehyde
MNXM245@MNXC3
N-acetyl-L-glutamate 5-semialdehyde
MNXM1062@MNXC3


In [6]:
model.metabolites.get_by_id('MNXM245@MNXC3') # L-glutamate 5-semialdehyde

Metabolite identifier,MNXM245@MNXC3
Name,L-glutamate 5-semialdehyde
Memory address,0x07fb37ca988d0
Formula,C5H9NO3
Compartment,MNXC3
In 3 reaction(s),"mnxr103352c3, mnxr153703c3, mnxr99896c3"


In [7]:
for metabolite in model.metabolites.query('L-glutamate', 'name'):
    print(metabolite.name)
    print(metabolite.id)

L-glutamate
MNXM741173@MNXC3
L-glutamate
MNXM741173@MNXC2
N-acetyl-L-glutamate
MNXM730@MNXC3
L-glutamate 5-semialdehyde
MNXM245@MNXC3
Tetrahydropteroyltri-L-glutamate
MNXM733072@MNXC3
5-Methyltetrahydropteroyltri-L-glutamate
MNXM733158@MNXC3
10-Formyl-THF-L-glutamate
MNXM33047@MNXC3
tetrahydropteroyltri-L-glutamate
MNXM733068@MNXC3
N-acetyl-L-glutamate 5-semialdehyde
MNXM1062@MNXC3


In [8]:
model.metabolites.get_by_id('MNXM741173@MNXC3') # L-glutamate

Metabolite identifier,MNXM741173@MNXC3
Name,L-glutamate
Memory address,0x07fb37cb792e8
Formula,C5H8NO4
Compartment,MNXC3
In 43 reaction(s),"mnxr96945c3, mnxr95843c3, mnxr104301c3, mnxr141897c3, mnxr100288c3, mnxr103225c3, mnxr100086c3, mnxr96230c3, mnxr100136c3, mnxr100811c3, mnxr96485c3, mnxr146193c3, mnxr108904c3, mnxr144995c3, mnxr1..."


In [9]:
for metabolite in model.metabolites.query('ornithine', 'name'):
    print(metabolite.name)
    print(metabolite.id)

L-ornithine
MNXM741175@MNXC3
N(2)-acetyl-L-ornithine
MNXM817@MNXC3


In [10]:
model.metabolites.get_by_id('MNXM741175@MNXC3') # L-ornithine

Metabolite identifier,MNXM741175@MNXC3
Name,L-ornithine
Memory address,0x07fb37cae9470
Formula,C5H13N2O2
Compartment,MNXC3
In 3 reaction(s),"mnxr102137c3, mnxr102218c3, mnxr102221c3"


In [11]:
for metabolite in model.metabolites.query('oxoglutarate', 'name'):
    print(metabolite.name)
    print(metabolite.id)

2-oxoglutarate
MNXM20@MNXC3


In [12]:
model.metabolites.get_by_id('MNXM20@MNXC3') # 2-oxoglutarate

Metabolite identifier,MNXM20@MNXC3
Name,2-oxoglutarate
Memory address,0x07fb37cb8b0f0
Formula,C5H4O5
Compartment,MNXC3
In 16 reaction(s),"mnxr105305c3, mnxr104301c3, mnxr103225c3, mnxr100086c3, mnxr96230c3, mnxr95852c3, mnxr146406c3, mnxr106443c3, mnxr95388c3, mnxr105000c3, mnxr105310c3, mnxr126234c3, mnxr96228c3, mnxr96229c3, mnxr10..."


In [13]:
new_reaction_1.add_metabolites({model.metabolites.get_by_id('MNXM245@MNXC3'): -1, # L-glutamate 5-semialdehyde
                              model.metabolites.get_by_id('MNXM741173@MNXC3'): -1, # L-glutamate 
                              model.metabolites.get_by_id('MNXM741175@MNXC3'): 1, # L-ornithine
                              model.metabolites.get_by_id('MNXM20@MNXC3'): 1 # 2-oxoglutarate
                             })

In [14]:
print(new_reaction_1.build_reaction_string())

MNXM245@MNXC3 + MNXM741173@MNXC3 --> MNXM20@MNXC3 + MNXM741175@MNXC3


In [15]:
model.add_reactions([new_reaction_1])

In [16]:
model.reactions.OAT

Reaction identifier,OAT
Name,
Memory address,0x07fb4440fe5f8
Stoichiometry,MNXM245@MNXC3 + MNXM741173@MNXC3 --> MNXM20@MNXC3 + MNXM741175@MNXC3 L-glutamate 5-semialdehyde + L-glutamate --> 2-oxoglutarate + L-ornithine
GPR,
Lower bound,0.0
Upper bound,1000.0


In [17]:
with model:
    model.objective = model.reactions.OAT
    print(model.optimize().objective_value)  # We produce L-ornithine

1000.0


We add bounds

In [18]:
model.add_boundary(model.metabolites.get_by_id('MNXM741175@MNXC3'), type='demand') # Demand for L-ornithine

Reaction identifier,DM_MNXM741175@MNXC3
Name,L-ornithine demand
Memory address,0x07fb4440feac8
Stoichiometry,MNXM741175@MNXC3 --> L-ornithine -->
GPR,
Lower bound,0
Upper bound,1000.0


## 4.1.1.17

In [19]:
new_reaction_2 = Reaction('ODC')  # ODC -> (o)rnithine (d)e(c)arboxylase

In [20]:
for metabolite in model.metabolites.query('putrescine', 'name'):
    print(metabolite.name)
    print(metabolite.id)

putrescine
MNXM118@MNXC3


In [21]:
model.metabolites.get_by_id('MNXM118@MNXC3') # Putrescine

Metabolite identifier,MNXM118@MNXC3
Name,putrescine
Memory address,0x07fb37c890400
Formula,C4H14N2
Compartment,MNXC3
In 1 reaction(s),mnxr95860c3


In [22]:
for metabolite in model.metabolites.query('CO2', 'name'):
    print(metabolite.name)
    print(metabolite.id)

CO2
MNXM13@MNXC3
CO2
MNXM13@MNXC2


In [23]:
model.metabolites.get_by_id('MNXM13@MNXC3') # CO2

Metabolite identifier,MNXM13@MNXC3
Name,CO2
Memory address,0x07fb37cb3e128
Formula,CO2
Compartment,MNXC3
In 63 reaction(s),"mnxr96142c3, mnxr106659c3, mnxr111054c3, mnxr95786c3, mnxr134885c3, mnxr103098c3, mnxr146718c3, mnxr103096c3, mnxr110387c3, mnxr134860c3, mnxr103067c3, mnxr96481c3, mnxr108735c3, mnxr102428c3, mnxr..."


In [24]:
new_reaction_2.add_metabolites({model.metabolites.get_by_id('MNXM741175@MNXC3'): -1, # L-ornithine
                              model.metabolites.get_by_id('MNXM13@MNXC3'): 1, # CO2
                              model.metabolites.get_by_id('MNXM118@MNXC3'): 1 # # Putrescine
                             })

In [25]:
print(new_reaction_2.build_reaction_string())

MNXM741175@MNXC3 --> MNXM118@MNXC3 + MNXM13@MNXC3


In [26]:
model.add_reactions([new_reaction_2])

In [27]:
model.reactions.ODC

Reaction identifier,ODC
Name,
Memory address,0x07fb37c412160
Stoichiometry,MNXM741175@MNXC3 --> MNXM118@MNXC3 + MNXM13@MNXC3 L-ornithine --> putrescine + CO2
GPR,
Lower bound,0.0
Upper bound,1000.0


In [28]:
with model:
    model.objective = model.reactions.OAT
    print("Maximum Putrescine yield: ", model.optimize().objective_value) # We produce putrescine

Maximum Putrescine yield:  1000.0


In [29]:
model.add_boundary(model.metabolites.get_by_id('MNXM118@MNXC3'), type='demand') # Demand for putrescine

Reaction identifier,DM_MNXM118@MNXC3
Name,putrescine demand
Memory address,0x07fb37c3f6b00
Stoichiometry,MNXM118@MNXC3 --> putrescine -->
GPR,
Lower bound,0
Upper bound,1000.0


The production of putrescine seems to just follow the upper bound. To investigate this further we looked into the fluxes in the original unmodified model

# Model fluxes check

In [30]:
modelcheck = read_sbml_model('seed_Seed196627_4.COBRA-sbml3.xml')


The model is checked for fluxes

In [31]:
solution = modelcheck.optimize()
solution.fluxes

mnxr01c2          0.0
mnxr01c3          0.0
mnxr02c2b         0.0
mnxr100018c2c3    0.0
mnxr100019c2b     0.0
                 ... 
mnxr99903c3       0.0
mnxr99909c3       0.0
reac6EA132C5     -0.0
reacCBCD7D74      0.0
reacD3FA036D      0.0
Name: fluxes, Length: 1007, dtype: float64

The fluxes look to be zero. We want to check if there are any fluxes that are not zero

In [32]:
solution.fluxes[solution.fluxes != 0]

Series([], Name: fluxes, dtype: float64)

In [35]:
# We look up the model summary. This shows that there are no fluxes
model.summary()

As this model seems to be incomplete, we will try and insert the corresponding pathway in another production host, E. coli. 